<a href="https://colab.research.google.com/github/victormurcia/VCHAMPS/blob/main/VCHAMPS_Construction_of_preHosp_and_Medication_Harmonizationipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#General utilities
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm  # Import tqdm for the progress bar
import math
import glob,shutil,os,warnings,math,time,sys,re
from typing import List
import dask.dataframe as dd
from dask.diagnostics import ProgressBar

import sqlite3

#For UUID generation
import uuid

#For Slider viz
import ipywidgets as widgets
from IPython.display import display, clear_output,HTML

#Enable data to be extracted and downloaded from my Google Drive
from google.colab import drive, files
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Specify the path to the desired directory
directory_path = r'/content/drive/MyDrive/VCHAMPS - Train Cleaned'

# Change the current working directory to the desired directory
os.chdir(directory_path)

# Verify the current working directory
cwd = os.getcwd()

print(f"Current working directory: {cwd}")

Current working directory: /content/drive/MyDrive/VCHAMPS - Train Cleaned


In [3]:
#Instantiate meds ordered to keep df
med_order_keep_df = pd.read_csv('/content/drive/MyDrive/keepMedOrders.csv')
med_order_keep_df = med_order_keep_df.drop('Unnamed: 0',axis=1)
#Drop medications where keep = 0
med_order_keep_df = med_order_keep_df[med_order_keep_df['keep'] == 1]
med_order_keep_df

,desc,keep,acei,arb,arni,sglt2i,mra,gdmtBB,loopDiuretic,thiazideDiuretic,...,vasopressin,enoxaparin,cilostazol,dpp4i,midodrine,nsaids,thiazolidinediones,chestPainRx,IMiDs,checkpointinhibitors
0,methotrexate ; valsartan,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,acetylsalicylic acid ; citric acid ; sodium bi...,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,trandolapril ; verapamil,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,acetylsalicylic acid ; clopidogrel,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,bumetanide ; other antipruritics ; salicylic a...,1,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
456,fosinopril ; lisinopril,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
457,enalapril ; felodipine,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
458,moexipril,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
459,cilazapril,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
#Instantiate meds ordered df
medications_ordered_df = dd.read_parquet('/content/drive/MyDrive/VCHAMPS - Train Cleaned-Mapped/medications_ordered/*.parquet')
medications_ordered_df = medications_ordered_df.compute()
medications_ordered_df

,Internalpatientid,Age at med ordered,Order date,Medication start date,Medication stop date,Order discontinue date,Ordered medication atc 5,Order status,Stop reason,Encounter ID
0,122253,54,2001-08-08 00:07:03,2001-08-07 12:08:03,2002-08-08 12:08:03,NaT,hypromellose,expired,Not specified (no value),f3d3e234-4d72-4eb8-84c8-55350ced635b
1,98801,63,2013-08-23 15:41:00,2013-09-15 23:08:00,2014-01-03 23:08:00,2014-01-04 17:04:00,docusate,discontinued,Not specified (no value),b8508771-ddd3-4b74-825c-f2fb6e5c4f06
2,61915,64,2011-08-02 01:21:00,2011-08-01 17:07:00,2012-08-01 17:07:00,NaT,furosemide,expired,Not specified (no value),a2a885ac-a147-4155-ab65-b8c3450f3a81
3,46322,43,2003-06-07 11:39:47,2003-06-06 17:07:47,2004-05-27 17:07:47,2004-07-11 05:16:47,thiamine (vitamin b1),discontinued,Not specified (no value),17815e91-05bc-4da4-8665-4392e8ed8acc
4,152671,60,2002-11-18 12:22:00,2002-11-18 12:22:00,2002-11-18 03:52:00,2002-11-18 12:23:00,codeine ; paracetamol (acetaminophen),cancelled,Not specified (no value),08fd283e-e3c4-46b5-a484-b091d29aebd5
...,...,...,...,...,...,...,...,...,...,...
9995,84208,84,2000-04-26 19:39:54,2000-04-27 06:15:54,2000-05-20 18:01:54,2000-05-20 18:01:54,ranitidine,discontinued,Not specified (no value),5c7b0bf7-9d7e-42b6-aa77-4599e42067ea
9996,87788,73,2000-12-03 17:50:01,2000-12-08 05:53:01,2001-09-23 05:53:01,2001-12-25 17:43:01,metformin,discontinued,Not specified (no value),02470d75-e47f-4dee-8797-582870933e3a
9997,91047,87,2005-07-28 14:44:08,2005-07-28 15:38:08,2005-08-04 03:51:08,NaT,ceftriaxone,expired,Not specified (no value),3920fffa-a56d-5b91-bfbf-905d83379d61
9998,99463,65,2012-07-13 19:50:38,2012-07-13 19:52:38,2012-07-14 18:45:38,2012-07-14 18:45:38,insulin (human) fast-acting,discontinued/edit,Not specified (no value),8884f27e-4608-52d9-a17a-42b486b13324


In [5]:
#Instantiate meds to keep list for filtering
meds_to_keep_list = med_order_keep_df['desc'].to_list()
len(meds_to_keep_list)

461

In [6]:
#Filter dataframe for meds to keep based on desc
medications_ordered_df = medications_ordered_df[medications_ordered_df['Ordered medication atc 5'].isin(meds_to_keep_list)]
medications_ordered_df

,Internalpatientid,Age at med ordered,Order date,Medication start date,Medication stop date,Order discontinue date,Ordered medication atc 5,Order status,Stop reason,Encounter ID
2,61915,64,2011-08-02 01:21:00,2011-08-01 17:07:00,2012-08-01 17:07:00,NaT,furosemide,expired,Not specified (no value),a2a885ac-a147-4155-ab65-b8c3450f3a81
7,167167,83,2009-08-06 05:56:27,2009-11-02 15:06:27,2009-11-03 15:06:27,NaT,isosorbide mononitrate,expired,Not specified (no value),867250ac-05d0-48e2-9966-f5e94da2734c
10,110822,90,2005-07-16 19:33:25,2005-07-16 11:58:25,2005-07-31 21:49:25,2005-07-31 21:49:25,acetylsalicylic acid,discontinued,Not specified (no value),11b48ad1-f439-5899-8730-522fab60cd5c
12,154425,66,2004-12-11 19:25:55,2004-11-13 19:53:55,2005-01-07 22:32:55,NaT,isosorbide dinitrate,expired,Not specified (no value),9cabf805-0cd7-59ae-ba0a-48d55aa33aab
15,77629,73,1999-11-22 13:04:36,1999-11-22 13:04:36,2001-11-06 17:34:36,1999-12-19 02:11:36,isosorbide dinitrate,discontinued,Not specified (no value),66c4fd73-d5b8-5099-9dea-f1645c8b9fac
...,...,...,...,...,...,...,...,...,...,...
9981,103238,82,2016-07-26 01:29:04,2016-07-26 10:46:04,2016-07-26 04:29:04,2016-07-26 04:29:04,insulin aspart fast-acting,cancelled,Cancelled by Requesting Physician,f4bfe023-25a3-5d4e-8a8e-87adc9ecfae0
9983,35089,84,2002-06-30 12:42:56,2002-06-30 12:52:56,2002-06-30 13:36:56,2002-06-30 13:36:56,dobutamine ; glucose,discontinued/edit,Not specified (no value),ba8210d3-4e2d-4349-97e9-5af9fa721bff
9989,162425,83,2001-12-15 08:41:08,2002-05-31 08:41:08,2002-12-16 08:41:08,NaT,metoprolol,expired,Not specified (no value),4f2b5cb1-e4a0-4069-bf55-dfa7a1b3f377
9998,99463,65,2012-07-13 19:50:38,2012-07-13 19:52:38,2012-07-14 18:45:38,2012-07-14 18:45:38,insulin (human) fast-acting,discontinued/edit,Not specified (no value),8884f27e-4608-52d9-a17a-42b486b13324


In [7]:
medications_ordered_df = pd.merge(medications_ordered_df, med_order_keep_df, left_on='Ordered medication atc 5', right_on='desc', how='inner')
#medications_ordered_df = medications_ordered_df.drop(['desc','keep'],axis=1)
medications_ordered_df

,Internalpatientid,Age at med ordered,Order date,Medication start date,Medication stop date,Order discontinue date,Ordered medication atc 5,Order status,Stop reason,Encounter ID,...,vasopressin,enoxaparin,cilostazol,dpp4i,midodrine,nsaids,thiazolidinediones,chestPainRx,IMiDs,checkpointinhibitors
0,61915,64,2011-08-02 01:21:00,2011-08-01 17:07:00,2012-08-01 17:07:00,NaT,furosemide,expired,Not specified (no value),a2a885ac-a147-4155-ab65-b8c3450f3a81,...,0,0,0,0,0,0,0,0,0,0
1,33870,71,2007-11-13 18:23:39,2007-11-13 18:23:39,2007-11-26 07:49:39,2007-11-26 20:33:39,furosemide,discontinued/edit,Not specified (no value),9b3f3f81-e4f3-464c-9223-dd7d8bd57138,...,0,0,0,0,0,0,0,0,0,0
2,48767,84,2019-09-28 04:32:51,2019-09-28 04:34:51,2019-09-28 05:13:51,NaT,furosemide,expired,Not specified (no value),42cc3283-ea83-516f-b70f-2a350af4ee97,...,0,0,0,0,0,0,0,0,0,0
3,124600,92,2004-07-14 19:05:57,2004-07-14 19:05:57,2004-07-15 19:05:57,NaT,furosemide,expired,Not specified (no value),b569f7a3-22fd-5cb1-a425-4a1ad5ce2cba,...,0,0,0,0,0,0,0,0,0,0
4,40729,48,2012-11-11 09:16:44,2012-11-11 09:52:44,2012-11-11 16:30:44,2012-11-11 16:30:44,furosemide,discontinued,Not specified (no value),58381482-5e6c-5a32-b896-362bceb6a945,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2769209,85129,76,2001-11-22 20:47:22,2001-11-22 20:47:22,2002-11-23 20:47:22,NaT,insulin (human) long-acting ; zinc,expired,Not specified (no value),75ff2567-d719-4056-b926-47ed10498490,...,0,0,0,0,0,0,0,0,0,0
2769210,115905,84,2009-11-01 14:17:27,2009-12-25 21:01:27,2010-12-24 21:01:27,2010-12-25 10:45:27,insulin detemir ; orphenadrine (citrate),discontinued,Not specified (no value),9d092042-9850-4c39-b679-fd77864c011d,...,0,0,0,0,0,0,0,0,0,0
2769211,99976,77,2018-03-08 01:17:20,2018-03-07 13:18:20,2018-06-21 03:10:20,2018-06-21 03:10:20,ertugliflozin,discontinued,Obsolete Order,268db7d2-57d8-4bd7-98de-8cb528d28c91,...,0,0,0,0,0,0,0,0,0,0
2769212,167678,81,2014-03-09 13:18:54,2014-03-10 20:27:54,2015-03-09 20:27:54,NaT,dimercaprol ; prednisolone,expired,Not specified (no value),6d017556-0ba4-437a-862f-8e3436df9c4d,...,0,0,0,0,0,0,0,0,0,0


In [8]:
medications_ordered_df.to_parquet('/content/drive/MyDrive/VCHAMPS - Final Train Data/medications_ordered.parquet')

In [4]:
#Instantiate meds ordered df
medications_ordered_df = dd.read_parquet('/content/drive/MyDrive/VCHAMPS - Final Train Data/medications_ordered.parquet')
medications_ordered_df = medications_ordered_df.compute()
medications_ordered_df

,Internalpatientid,Age at med ordered,Order date,Medication start date,Medication stop date,Order discontinue date,Ordered medication atc 5,Order status,Stop reason,Encounter ID,...,vasopressin,enoxaparin,cilostazol,dpp4i,midodrine,nsaids,thiazolidinediones,chestPainRx,IMiDs,checkpointinhibitors
0,61915,64,2011-08-02 01:21:00,2011-08-01 17:07:00,2012-08-01 17:07:00,NaT,furosemide,expired,Not specified (no value),a2a885ac-a147-4155-ab65-b8c3450f3a81,...,0,0,0,0,0,0,0,0,0,0
1,33870,71,2007-11-13 18:23:39,2007-11-13 18:23:39,2007-11-26 07:49:39,2007-11-26 20:33:39,furosemide,discontinued/edit,Not specified (no value),9b3f3f81-e4f3-464c-9223-dd7d8bd57138,...,0,0,0,0,0,0,0,0,0,0
2,48767,84,2019-09-28 04:32:51,2019-09-28 04:34:51,2019-09-28 05:13:51,NaT,furosemide,expired,Not specified (no value),42cc3283-ea83-516f-b70f-2a350af4ee97,...,0,0,0,0,0,0,0,0,0,0
3,124600,92,2004-07-14 19:05:57,2004-07-14 19:05:57,2004-07-15 19:05:57,NaT,furosemide,expired,Not specified (no value),b569f7a3-22fd-5cb1-a425-4a1ad5ce2cba,...,0,0,0,0,0,0,0,0,0,0
4,40729,48,2012-11-11 09:16:44,2012-11-11 09:52:44,2012-11-11 16:30:44,2012-11-11 16:30:44,furosemide,discontinued,Not specified (no value),58381482-5e6c-5a32-b896-362bceb6a945,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2769209,85129,76,2001-11-22 20:47:22,2001-11-22 20:47:22,2002-11-23 20:47:22,NaT,insulin (human) long-acting ; zinc,expired,Not specified (no value),75ff2567-d719-4056-b926-47ed10498490,...,0,0,0,0,0,0,0,0,0,0
2769210,115905,84,2009-11-01 14:17:27,2009-12-25 21:01:27,2010-12-24 21:01:27,2010-12-25 10:45:27,insulin detemir ; orphenadrine (citrate),discontinued,Not specified (no value),9d092042-9850-4c39-b679-fd77864c011d,...,0,0,0,0,0,0,0,0,0,0
2769211,99976,77,2018-03-08 01:17:20,2018-03-07 13:18:20,2018-06-21 03:10:20,2018-06-21 03:10:20,ertugliflozin,discontinued,Obsolete Order,268db7d2-57d8-4bd7-98de-8cb528d28c91,...,0,0,0,0,0,0,0,0,0,0
2769212,167678,81,2014-03-09 13:18:54,2014-03-10 20:27:54,2015-03-09 20:27:54,NaT,dimercaprol ; prednisolone,expired,Not specified (no value),6d017556-0ba4-437a-862f-8e3436df9c4d,...,0,0,0,0,0,0,0,0,0,0


In [5]:
inpatient_admissions_df = pd.read_parquet('/content/drive/MyDrive/VCHAMPS - Final Train Data/inpatient_admissions.parquet')
inpatient_admissions_df

,Internalpatientid,Encounter ID,Age at admission,Admission date,Discharge date,Admitting unit service,Discharging unit service,Admitting specialty,Discharging specialty,mortality_inhosp_allcause,...,pre_hosp_any,pre_hosp_cv,readmit_allcause_30d,readmit_allcause_90d,readmit_allcause_180d,readmit_allcause_365d,readmit_CV_30d,readmit_CV_90d,readmit_CV_180d,readmit_CV_365d
32196,1,0ce62d49-ea64-5941-b70c-89b4a1c4cc8a,79,2022-12-31 05:41:51,2023-01-01 23:06:30,MEDICINE,MEDICINE,DERMATOLOGY,GENERAL(ACUTE MEDICINE),0,...,0,0,0,0,0,1,0,0,0,0
32197,1,0ce62d49-ea64-5941-b70c-89b4a1c4cc8a,79,2022-12-31 05:41:51,2023-01-01 23:06:30,MEDICINE,MEDICINE,DERMATOLOGY,GENERAL(ACUTE MEDICINE),0,...,1,0,0,0,0,1,0,0,0,0
45845,1,52d5e3bc-aced-53a4-b8a4-4a458e55601f,68,2012-08-10 23:27:47,2012-08-14 20:57:06,MEDICINE,MEDICINE,PERIPHERAL VASCULAR,GENERAL(ACUTE MEDICINE),0,...,2,0,0,0,0,1,0,0,0,0
45846,1,52d5e3bc-aced-53a4-b8a4-4a458e55601f,68,2012-08-10 23:27:47,2012-08-14 20:57:06,MEDICINE,MEDICINE,PERIPHERAL VASCULAR,GENERAL(ACUTE MEDICINE),0,...,3,0,0,0,0,1,0,0,0,0
45847,1,aee47e87-cab8-5ca7-9947-21cb2daf476b,72,2016-07-16 15:18:16,2016-07-22 21:36:24,SURGERY,SURGERY,SPINAL CORD INJURY,UROLOGY,0,...,4,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
435812,169062,e6460fae-0e3b-5fda-a016-f36cdd654819,73,2004-08-11 21:52:41,2004-08-14 23:42:41,MEDICINE,MEDICINE,"PULMONARY, TUBERCULOSIS",Not specified (no value),0,...,15,0,1,1,1,1,0,0,0,0
910894,169062,f6956488-7e4c-5967-a95f-208aae167c25,73,2004-08-10 14:07:29,2004-08-11 21:53:50,NON-COUNT,NON-COUNT,SURGICAL STEPDOWN,Not specified (no value),0,...,16,0,1,1,1,1,0,0,0,0
910895,169062,f6956488-7e4c-5967-a95f-208aae167c25,73,2004-08-10 14:07:29,2004-08-11 21:53:50,NON-COUNT,NON-COUNT,SURGICAL STEPDOWN,Not specified (no value),0,...,17,0,1,1,1,1,0,0,0,0
432541,169064,0c52ac5b-8578-50a3-9f26-fbdf68dad075,82,2008-08-19 03:12:47,2008-08-26 07:34:10,SURGERY,SURGERY,NH SHORT STAY SKILLED NURSING,GENERAL SURGERY,0,...,0,0,-1,-1,-1,-1,0,0,0,0


In [6]:
# Set the maximum number of rows to be displayed to 100
medications_ordered_df.dtypes

Internalpatientid                 int32
Age at med ordered                 int8
Order date               datetime64[ns]
Medication start date    datetime64[ns]
Medication stop date     datetime64[ns]
                              ...      
nsaids                            int64
thiazolidinediones                int64
chestPainRx                       int64
IMiDs                             int64
checkpointinhibitors              int64
Length: 69, dtype: object

In [7]:
#Let me convert some stuff to int8, might help with merging
columns_to_convert = ['keep', 'acei', 'arb', 'arni', 'sglt2i', 'mra', 'gdmtBB', 'loopDiuretic', 'thiazideDiuretic', 'p2y12i', 'aspirin', 'doac', 'warfarin', 'amio', 'sotalol', 'dofetilide', 'dobutamine', 'dopamine', 'milrinone', 'mexiletine', 'dronedarone', 'antiarrhythmic1c', 'nonDihydropyridineCAB', 'bicarbonate', 'phosBinder', 'he_ppx', 'asa5', 'immunoMod', 'methotrexate', 'tnfA_inh', 'cni', 'mTor_inh', 'prednisone', 'nicotine', 'parkinsons', 'pHTN_Rx', 'insulin', 'alzheimers_Rx', 'methadone', 'oxygen', 'hcq', 'rituximab', 'cyclophosphamide', 'doxorubicin', 'cisplatin', 'pyrimidineAM', 'topoisomerasei', 'norepinephrine', 'vasopressin', 'enoxaparin', 'cilostazol', 'dpp4i', 'midodrine', 'nsaids', 'thiazolidinediones', 'chestPainRx', 'IMiDs', 'checkpointinhibitors']

medications_ordered_df[columns_to_convert] = medications_ordered_df[columns_to_convert].astype('int8')
medications_ordered_df.dtypes

Internalpatientid                 int32
Age at med ordered                 int8
Order date               datetime64[ns]
Medication start date    datetime64[ns]
Medication stop date     datetime64[ns]
                              ...      
nsaids                             int8
thiazolidinediones                 int8
chestPainRx                        int8
IMiDs                              int8
checkpointinhibitors               int8
Length: 69, dtype: object

# Determining Medications Patient Was Taking At Time of Admission


In [10]:
# Drop any rows that have nans on the start date in medications_ordered_df
medications_ordered_df.dropna(subset=['Medication start date'], inplace=True)

# Sort both DataFrames by the relevant date columns
inpatient_admissions_df.sort_values('Admission date', inplace=True)
medications_ordered_df.sort_values('Medication start date', inplace=True)

# Create an in-memory SQLite database connection
conn = sqlite3.connect(':memory:')

# Write the DataFrames to the SQLite database as temporary tables
inpatient_admissions_df.to_sql('inpatient_admissions', conn, index=False)
medications_ordered_df.to_sql('medications_ordered', conn, index=False)

# SQL query to perform the LEFT JOIN based on the specified conditions
sql_query = '''
SELECT *
FROM inpatient_admissions AS ia
LEFT JOIN medications_ordered AS mo
ON ia.Internalpatientid = mo.Internalpatientid
    AND ia.[Admission date] > mo.[Medication start date]
    AND ia.[Admission date] < mo.[Medication stop date];
'''

# Execute the SQL query and read the result into a new DataFrame
merged_df = pd.read_sql_query(sql_query, conn)

# Close the database connection
conn.close()

In [11]:
merged_df

,Internalpatientid,Encounter ID,Age at admission,Admission date,Discharge date,Admitting unit service,Discharging unit service,Admitting specialty,Discharging specialty,mortality_inhosp_allcause,...,vasopressin,enoxaparin,cilostazol,dpp4i,midodrine,nsaids,thiazolidinediones,chestPainRx,IMiDs,checkpointinhibitors
0,167101,1dcb3735-6d85-5dcf-8ad3-65c69524cfd7,58,1990-08-07 10:26:37,1998-09-26 10:37:10,NHCU,MEDICINE,POLYTRAUMA REHAB UNIT,NHCU,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,167101,1dcb3735-6d85-5dcf-8ad3-65c69524cfd7,58,1990-08-07 10:26:37,1998-09-26 10:37:10,NHCU,MEDICINE,POLYTRAUMA REHAB UNIT,NHCU,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,104446,2c0643ec-65da-5130-8723-51de869a7138,54,1992-09-19 18:43:17,1997-12-30 00:34:28,DOMICILIARY,DOMICILIARY,DOMICILIARY,DOMICILIARY,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,74955,3a012aae-f11b-522f-8937-e140e97d98e9,87,1992-10-28 19:12:22,1996-04-01 02:19:06,NHCU,MEDICINE,Not specified (no value),Not specified (no value),1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,74955,3a012aae-f11b-522f-8937-e140e97d98e9,87,1992-10-28 19:12:22,1996-04-01 02:19:06,NHCU,MEDICINE,Not specified (no value),Not specified (no value),1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1281366,43617,77291074-e6af-5d5e-8237-98b311deaec1,91,2026-03-16 01:31:33,2026-03-18 18:15:57,MEDICINE,MEDICINE,NH SHORT STAY RESTORATIVE,GENERAL(ACUTE MEDICINE),0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1281367,43617,77291074-e6af-5d5e-8237-98b311deaec1,91,2026-03-16 01:31:33,2026-03-18 18:15:57,MEDICINE,MEDICINE,NH SHORT STAY RESTORATIVE,GENERAL(ACUTE MEDICINE),0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1281368,43617,77291074-e6af-5d5e-8237-98b311deaec1,91,2026-03-16 01:31:33,2026-03-18 18:15:57,MEDICINE,MEDICINE,NH SHORT STAY RESTORATIVE,GENERAL(ACUTE MEDICINE),0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1281369,66879,211289b3-d7cb-5457-8743-394cca1807b2,102,2026-03-22 20:34:18,2026-04-02 18:38:09,SURGERY,MEDICINE,SUBST ABUSE CWT/TRANS RESID,CARDIOLOGY,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
for col in merged_df.columns:
  print(col)

Internalpatientid
Encounter ID
Age at admission
Admission date
Discharge date
Admitting unit service
Discharging unit service
Admitting specialty
Discharging specialty
mortality_inhosp_allcause
Outpatientreferralflag
Agentorangeflag
CV diagnosis
diagnosis
code
cc Status_CC
cc Status_MCC
cc Status_NCC
Discharge disposition_Death with autopsy
Discharge disposition_Death without autopsy
Discharge disposition_Irregular
Discharge disposition_NBC or while ASIH
Discharge disposition_Regular
Discharge disposition_Transfer
rehosp_allcause
Smallest Time Passed
mortality_inhosp_CV
length_of_stay
pre_hosp_any
pre_hosp_cv
readmit_allcause_30d
readmit_allcause_90d
readmit_allcause_180d
readmit_allcause_365d
readmit_CV_30d
readmit_CV_90d
readmit_CV_180d
readmit_CV_365d
Internalpatientid
Age at med ordered
Order date
Medication start date
Medication stop date
Order discontinue date
Ordered medication atc 5
Order status
Stop reason
Encounter ID
desc
keep
acei
arb
arni
sglt2i
mra
gdmtBB
loopDiuretic
thi